# Part VI: User Defined Functions

This notebook looks at our client data *(please note this is not real data)* and shows how complex logic can be created in Python, and then applied to a dataframe as a udf.

In this scenario we are looking at data quality. Our sales team has noticed that when client emails have uncommon top-level domains, they are unable to concact them quickly. Emails ending in .com or .org have faster response times based on the research conducted by the sales team analysts. 

To help with this, we will flag any rows of data that do not end with .com or .org as invalid.

#### Load Data

We already have our mock client data in DBFS from earlier. Now we just need to load it into our notebook.

In [0]:
# File location and type
file_location = "/FileStore/tables/MOCK_CLIENT_DATA-1.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# Create Dataframe.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df.show(10, False)

+---+------------+---------+------------------------+---------------+--------------+---------------+---------------+---------------+
|id |first_name  |last_name|email                   |account_balance|average_return|portfolio_theme|trade_frequency|last_contact_dt|
+---+------------+---------+------------------------+---------------+--------------+---------------+---------------+---------------+
|1  |Bria        |Ruse     |bruse0@lulu.com         |$558243.81     |-4.53         |core           |Weekly         |2022-11-25     |
|2  |Merry       |Effemy   |meffemy1@paypal.com     |$3739.01       |8.09          |smart-beta     |Seldom         |2022-07-20     |
|3  |Chrisse     |Iannello |ciannello2@tinypic.com  |$347550.67     |-3.58         |smart-beta     |Often          |2022-12-16     |
|4  |Lorettalorna|Bonnette |lbonnette3@dropbox.com  |$250191.19     |4.37          |core           |Yearly         |2022-05-12     |
|5  |Maxwell     |Spellicy |mspellicy4@freewebs.com |$705098.95     |

Let's create a user-defined function (UDF) that will look at the email column and only consider it a valid email if it ends in ".com" or ".org". 

Notice that this python function is tagged with '@udf'

In [0]:
#create a udf that slices off the text after the 'dot' character
@udf
def checkEmail(email):
    validDomains = ["com","org"]
    arr = email.split(".")
    arrEnd = arr[-1]
    if arrEnd not in validDomains:
        return "Invalid domain"
    else:
        return "Valid domain"


In [0]:
# create new column that applies our udf to each row of the email column
df.withColumn("email_check", checkEmail("email")) \
    .show(10, truncate=False)

+---+------------+---------+------------------------+---------------+--------------+---------------+---------------+---------------+--------------+
|id |first_name  |last_name|email                   |account_balance|average_return|portfolio_theme|trade_frequency|last_contact_dt|email_check   |
+---+------------+---------+------------------------+---------------+--------------+---------------+---------------+---------------+--------------+
|1  |Bria        |Ruse     |bruse0@lulu.com         |$558243.81     |-4.53         |core           |Weekly         |2022-11-25     |Valid domain  |
|2  |Merry       |Effemy   |meffemy1@paypal.com     |$3739.01       |8.09          |smart-beta     |Seldom         |2022-07-20     |Valid domain  |
|3  |Chrisse     |Iannello |ciannello2@tinypic.com  |$347550.67     |-3.58         |smart-beta     |Often          |2022-12-16     |Valid domain  |
|4  |Lorettalorna|Bonnette |lbonnette3@dropbox.com  |$250191.19     |4.37          |core           |Yearly      

Great! We can see in the sample that our udf checked each email and found the .tv entry and flagged it as invalid.

While this is a very simple sample and there are many scenarios where the logic implemented here would not hold up, you should now have a better understanding of how to create and apply udf's to your data. 

Thanks for reading!